In [1]:
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy
from spacy.training import Example
from spacy.scorer import Scorer
import re
import os
from spacy.language import Language
from prettytable import PrettyTable
from IPython.display import clear_output
from collections import Counter

In [2]:
ENTITIES = ["NOMBRE", "APELLIDO", "DNI", "DIRECCION", "CIUDAD", "PROVINCIA", "CP", "TELEFONO", "REF_CATASTRAL", "SEGURIDAD_SOCIAL", "CUENTA_BANCARIA", "EMAIL", "MATRICULA", "CSV", "URL"]
nlp = spacy.blank("es")

In [3]:
def cargarDocAnotado(ficheroAnotado):
        
    palabras, entidades = zip(* [linea.rstrip().split("\t") for linea in open(ficheroAnotado, "r", encoding = "utf-8") if linea != "\n"] )
    
    palabras, entidades = list(palabras), list(entidades)
    
    doc = Doc(nlp.vocab, words = palabras, ents = entidades)
            
    return doc

def cargarTextoDocumento(fichero):
    with open(fichero, 'r', encoding='utf-8') as file:
        text = ' '.join([x.rstrip() for x in file])
    return text        

## Apariciones de las menciones en documentos y tipo de entidad (Global)

In [4]:
path_doc_anotados = "annotated/"
anotados_list = os.listdir(path_doc_anotados)
anotados_list = [x for x in anotados_list if re.match(r"^doc_*", x)]

In [5]:
path_Probar = "test/"
lista_probar = os.listdir(path_Probar)
lista_probar = [x for x in lista_probar if re.match(r"^doc_*", x)]

In [6]:
lista_documentos = [x.replace(".spacy", ".tsv") for x in lista_probar if x.replace(".spacy", ".tsv") in anotados_list]
len(lista_documentos)

62

In [7]:
lista_documentos = anotados_list
lista_documentos[:2]

['doc_127.txt.tsv', 'doc_137.txt.tsv']

In [8]:
contador_entidades_anotadas = {entidad: 0 for entidad in ENTITIES}
contador_documentos = {entidad: 0 for entidad in ENTITIES}

def incrementarContadorEntidades(contador_global, entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        contador_global[entidad] = contador_global[entidad] + repeticiones
        
def incrementarContadorDocumentos(entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        if repeticiones > 0:
            contador_documentos[entidad] = contador_documentos[entidad] + 1
    
for indice, documento in enumerate( lista_documentos ):
    clear_output(wait=True)
    print(f"Procesando Documento {indice + 1} de {len(lista_documentos)}")
    
    doc_anotado = cargarDocAnotado(f"{path_doc_anotados}{documento}")

    entidades_anotadas = Counter( [token.ent_type_ for token in doc_anotado if token.ent_iob_ == "B"] )

    incrementarContadorEntidades(contador_global = contador_entidades_anotadas, entidades_counter = entidades_anotadas)
    
    incrementarContadorDocumentos(entidades_counter = entidades_anotadas)

Procesando Documento 309 de 309


In [9]:
tabla = PrettyTable(["ENTIDAD", "DOCUMENTOS", "APARICIONES", "%DOC", "%APARICIONES"])

def calcularPorcentaje(num1, num2):
    try:
        porcentaje = (num1 / num2) * 100
        return round( porcentaje, 2 )
    except ZeroDivisionError:
        return 0

for ((k1, v1), (k2, v2)) in zip(contador_documentos.items(), contador_entidades_anotadas.items()):
    tabla.add_row( [k1, v1, v2, calcularPorcentaje(v1, 309), calcularPorcentaje(v2, 10774)] ) 

print("Número de Documentos y Apariciones por tipo de entidad")
print("=========================================================")
print(tabla)

Número de Documentos y Apariciones por tipo de entidad
+------------------+------------+-------------+-------+--------------+
|     ENTIDAD      | DOCUMENTOS | APARICIONES |  %DOC | %APARICIONES |
+------------------+------------+-------------+-------+--------------+
|      NOMBRE      |    309     |     2263    | 100.0 |     21.0     |
|     APELLIDO     |    309     |     4526    | 100.0 |    42.01     |
|       DNI        |     79     |     710     | 25.57 |     6.59     |
|    DIRECCION     |    160     |     323     | 51.78 |     3.0      |
|      CIUDAD      |    309     |     1994    | 100.0 |    18.51     |
|    PROVINCIA     |     5      |      7      |  1.62 |     0.06     |
|        CP        |    119     |     141     | 38.51 |     1.31     |
|     TELEFONO     |    117     |     118     | 37.86 |     1.1      |
|  REF_CATASTRAL   |     10     |      36     |  3.24 |     0.33     |
| SEGURIDAD_SOCIAL |     0      |      0      |  0.0  |     0.0      |
| CUENTA_BANCARIA  |  

## Apariciones de las menciones en documentos y tipo de entidad (Por Subconjunto de datos)

### Conjunto de Train

In [10]:
path_doc_anotados = "annotated/"
anotados_list = os.listdir(path_doc_anotados)
anotados_list = [x for x in anotados_list if re.match(r"^doc_*", x)]

In [11]:
path_Probar = "train/"
lista_probar = os.listdir(path_Probar)
lista_probar = [x for x in lista_probar if re.match(r"^doc_*", x)]

In [12]:
lista_documentos = [x.replace(".spacy", ".tsv") for x in lista_probar if x.replace(".spacy", ".tsv") in anotados_list]
len(lista_documentos)

216

In [13]:
contador_entidades_anotadas = {entidad: 0 for entidad in ENTITIES}
contador_documentos = {entidad: 0 for entidad in ENTITIES}

def incrementarContadorEntidades(contador_global, entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        contador_global[entidad] = contador_global[entidad] + repeticiones
        
def incrementarContadorDocumentos(entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        if repeticiones > 0:
            contador_documentos[entidad] = contador_documentos[entidad] + 1
    
for indice, documento in enumerate( lista_documentos ):
    clear_output(wait=True)
    print(f"Procesando Documento {indice + 1} de {len(lista_documentos)}")
    
    doc_anotado = cargarDocAnotado(f"{path_doc_anotados}{documento}")

    entidades_anotadas = Counter( [token.ent_type_ for token in doc_anotado if token.ent_iob_ == "B"] )

    incrementarContadorEntidades(contador_global = contador_entidades_anotadas, entidades_counter = entidades_anotadas)
    
    incrementarContadorDocumentos(entidades_counter = entidades_anotadas)

Procesando Documento 216 de 216


In [14]:
tabla = PrettyTable(["ENTIDAD", "DOCUMENTOS", "APARICIONES"])

def calcularPorcentaje(num1, num2):
    try:
        porcentaje = (num1 / num2) * 100
        return round( porcentaje, 2 )
    except ZeroDivisionError:
        return 0

for ((k1, v1), (k2, v2)) in zip(contador_documentos.items(), contador_entidades_anotadas.items()):
    tabla.add_row( [k1, v1, v2]) 

print("Número de Documentos y Apariciones por tipo de entidad (Conjunto de Train)")
print("============================================================================")
print(tabla)

Número de Documentos y Apariciones por tipo de entidad (Conjunto de Train)
+------------------+------------+-------------+
|     ENTIDAD      | DOCUMENTOS | APARICIONES |
+------------------+------------+-------------+
|      NOMBRE      |    216     |     1553    |
|     APELLIDO     |    216     |     3102    |
|       DNI        |     51     |     458     |
|    DIRECCION     |    107     |     191     |
|      CIUDAD      |    216     |     1397    |
|    PROVINCIA     |     3      |      5      |
|        CP        |     78     |      91     |
|     TELEFONO     |     77     |      78     |
|  REF_CATASTRAL   |     7      |      19     |
| SEGURIDAD_SOCIAL |     0      |      0      |
| CUENTA_BANCARIA  |     12     |      35     |
|      EMAIL       |     2      |      2      |
|    MATRICULA     |     4      |      4      |
|       CSV        |    215     |     215     |
|       URL        |    195     |     195     |
+------------------+------------+-------------+


### Conjunto de Validación

In [15]:
path_doc_anotados = "annotated/"
anotados_list = os.listdir(path_doc_anotados)
anotados_list = [x for x in anotados_list if re.match(r"^doc_*", x)]

In [16]:
path_Probar = "valid/"
lista_probar = os.listdir(path_Probar)
lista_probar = [x for x in lista_probar if re.match(r"^doc_*", x)]

In [17]:
lista_documentos = [x.replace(".spacy", ".tsv") for x in lista_probar if x.replace(".spacy", ".tsv") in anotados_list]
len(lista_documentos)

31

In [18]:
contador_entidades_anotadas = {entidad: 0 for entidad in ENTITIES}
contador_documentos = {entidad: 0 for entidad in ENTITIES}

def incrementarContadorEntidades(contador_global, entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        contador_global[entidad] = contador_global[entidad] + repeticiones
        
def incrementarContadorDocumentos(entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        if repeticiones > 0:
            contador_documentos[entidad] = contador_documentos[entidad] + 1
    
for indice, documento in enumerate( lista_documentos ):
    clear_output(wait=True)
    print(f"Procesando Documento {indice + 1} de {len(lista_documentos)}")
    
    doc_anotado = cargarDocAnotado(f"{path_doc_anotados}{documento}")

    entidades_anotadas = Counter( [token.ent_type_ for token in doc_anotado if token.ent_iob_ == "B"] )

    incrementarContadorEntidades(contador_global = contador_entidades_anotadas, entidades_counter = entidades_anotadas)
    
    incrementarContadorDocumentos(entidades_counter = entidades_anotadas)

Procesando Documento 31 de 31


In [19]:
tabla = PrettyTable(["ENTIDAD", "DOCUMENTOS", "APARICIONES"])

def calcularPorcentaje(num1, num2):
    try:
        porcentaje = (num1 / num2) * 100
        return round( porcentaje, 2 )
    except ZeroDivisionError:
        return 0

for ((k1, v1), (k2, v2)) in zip(contador_documentos.items(), contador_entidades_anotadas.items()):
    tabla.add_row( [k1, v1, v2]) 

print("Número de Documentos y Apariciones por tipo de entidad (Conjunto de Validacion)")
print("============================================================================")
print(tabla)

Número de Documentos y Apariciones por tipo de entidad (Conjunto de Validacion)
+------------------+------------+-------------+
|     ENTIDAD      | DOCUMENTOS | APARICIONES |
+------------------+------------+-------------+
|      NOMBRE      |     31     |     244     |
|     APELLIDO     |     31     |     492     |
|       DNI        |     10     |     102     |
|    DIRECCION     |     20     |      52     |
|      CIUDAD      |     31     |     218     |
|    PROVINCIA     |     1      |      1      |
|        CP        |     16     |      17     |
|     TELEFONO     |     16     |      16     |
|  REF_CATASTRAL   |     1      |      5      |
| SEGURIDAD_SOCIAL |     0      |      0      |
| CUENTA_BANCARIA  |     2      |      6      |
|      EMAIL       |     0      |      0      |
|    MATRICULA     |     1      |      1      |
|       CSV        |     31     |      31     |
|       URL        |     28     |      28     |
+------------------+------------+-------------+


### Conjunto de Test

In [20]:
path_doc_anotados = "annotated/"
anotados_list = os.listdir(path_doc_anotados)
anotados_list = [x for x in anotados_list if re.match(r"^doc_*", x)]

In [21]:
path_Probar = "test/"
lista_probar = os.listdir(path_Probar)
lista_probar = [x for x in lista_probar if re.match(r"^doc_*", x)]

In [22]:
lista_documentos = [x.replace(".spacy", ".tsv") for x in lista_probar if x.replace(".spacy", ".tsv") in anotados_list]
len(lista_documentos)

62

In [23]:
contador_entidades_anotadas = {entidad: 0 for entidad in ENTITIES}
contador_documentos = {entidad: 0 for entidad in ENTITIES}

def incrementarContadorEntidades(contador_global, entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        contador_global[entidad] = contador_global[entidad] + repeticiones
        
def incrementarContadorDocumentos(entidades_counter):
    for entidad, repeticiones in entidades_counter.items():
        if repeticiones > 0:
            contador_documentos[entidad] = contador_documentos[entidad] + 1
    
for indice, documento in enumerate( lista_documentos ):
    clear_output(wait=True)
    print(f"Procesando Documento {indice + 1} de {len(lista_documentos)}")
    
    doc_anotado = cargarDocAnotado(f"{path_doc_anotados}{documento}")

    entidades_anotadas = Counter( [token.ent_type_ for token in doc_anotado if token.ent_iob_ == "B"] )

    incrementarContadorEntidades(contador_global = contador_entidades_anotadas, entidades_counter = entidades_anotadas)
    
    incrementarContadorDocumentos(entidades_counter = entidades_anotadas)

Procesando Documento 62 de 62


In [24]:
tabla = PrettyTable(["ENTIDAD", "DOCUMENTOS", "APARICIONES"])

def calcularPorcentaje(num1, num2):
    try:
        porcentaje = (num1 / num2) * 100
        return round( porcentaje, 2 )
    except ZeroDivisionError:
        return 0

for ((k1, v1), (k2, v2)) in zip(contador_documentos.items(), contador_entidades_anotadas.items()):
    tabla.add_row( [k1, v1, v2]) 

print("Número de Documentos y Apariciones por tipo de entidad (Conjunto de Test)")
print("============================================================================")
print(tabla)

Número de Documentos y Apariciones por tipo de entidad (Conjunto de Test)
+------------------+------------+-------------+
|     ENTIDAD      | DOCUMENTOS | APARICIONES |
+------------------+------------+-------------+
|      NOMBRE      |     62     |     466     |
|     APELLIDO     |     62     |     932     |
|       DNI        |     18     |     150     |
|    DIRECCION     |     33     |      80     |
|      CIUDAD      |     62     |     379     |
|    PROVINCIA     |     1      |      1      |
|        CP        |     25     |      33     |
|     TELEFONO     |     24     |      24     |
|  REF_CATASTRAL   |     2      |      12     |
| SEGURIDAD_SOCIAL |     0      |      0      |
| CUENTA_BANCARIA  |     3      |      15     |
|      EMAIL       |     1      |      1      |
|    MATRICULA     |     1      |      2      |
|       CSV        |     62     |      62     |
|       URL        |     59     |      59     |
+------------------+------------+-------------+
